# Day15_1: ML 종합 프로젝트 - 정답 노트북

---

## 환경 설정

In [ ]:
# 필수 라이브러리 임포트
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, confusion_matrix, classification_report
)
from sklearn.pipeline import Pipeline

try:
    from xgboost import XGBClassifier
    XGBOOST_AVAILABLE = True
except ImportError:
    XGBOOST_AVAILABLE = False

import warnings
warnings.filterwarnings('ignore')

# 데이터 로드
df = pd.read_csv('datasets/ecommerce_churn.csv')
print(f"데이터 로드 완료: {df.shape}")

---

## Q1. 프로젝트 정의 작성

**문제**: 아래 빈칸을 채워 "중고차 가격 예측" 프로젝트 정의서를 완성하세요.

In [ ]:
# 정답 코드
project = {
    "프로젝트명": "중고차 가격 예측 시스템",
    "ML_문제_유형": "회귀 (Regression)",
    "평가_지표": ["RMSE (주요)", "MAE", "R2 Score"],
    "성공_기준": "RMSE <= 500만원"
}

print("중고차 가격 예측 프로젝트 정의서")
print("=" * 40)
for key, value in project.items():
    print(f"{key}: {value}")

In [ ]:
# 테스트
assert project["ML_문제_유형"] == "회귀 (Regression)", "회귀 문제입니다"
assert "RMSE" in str(project["평가_지표"]), "RMSE가 포함되어야 합니다"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
- 중고차 가격은 연속형 수치이므로 **회귀(Regression)** 문제
- 회귀 문제의 대표 평가 지표: RMSE, MAE, R2
- 성공 기준은 비즈니스 요구사항에 맞게 설정 (예: 500만원 이내 오차)

**핵심 개념**:
- 분류 vs 회귀: 타겟이 범주형이면 분류, 연속형이면 회귀
- 평가 지표 선택: 문제 유형에 맞는 지표 선택 필수

**실무 팁**:
- 프로젝트 정의서는 이해관계자와 합의 필수
- 성공 기준은 비즈니스 임팩트 고려

---

## Q2. 결측치 확인

**문제**: df 데이터프레임의 결측치를 컬럼별로 확인하고, 결측치가 있는 컬럼만 출력하세요.

In [ ]:
# 정답 코드
missing = df.isnull().sum()
missing_cols = missing[missing > 0]

if len(missing_cols) > 0:
    print("결측치가 있는 컬럼:")
    print(missing_cols)
else:
    print("결측치가 없습니다!")
    
print(f"\n전체 결측치 수: {df.isnull().sum().sum()}")

In [ ]:
# 테스트
total_missing = df.isnull().sum().sum()
print(f"검증: 전체 결측치 = {total_missing}")

### 풀이 설명

**접근 방법**:
1. `df.isnull().sum()`으로 컬럼별 결측치 수 계산
2. 결측치가 0보다 큰 컬럼만 필터링
3. 조건문으로 결과 분기 처리

**핵심 개념**:
- `isnull()`: 결측치 여부를 Boolean으로 반환
- Boolean 인덱싱: `series[series > 0]`

**대안**:
```python
# 비율도 함께 확인
missing_pct = (df.isnull().sum() / len(df) * 100).round(2)
```

---

## Q3. 클래스 불균형 확인

**문제**: 타겟 변수 'churned'의 클래스별 비율을 계산하고, 불균형 비율을 출력하세요.

In [ ]:
# 정답 코드
# 클래스별 개수
class_counts = df['churned'].value_counts()

# 클래스별 비율
class_ratio = df['churned'].value_counts(normalize=True) * 100

# 불균형 비율 계산
imbalance_ratio = class_counts[0] / class_counts[1]

print("타겟 변수 분포:")
print(f"  유지 (0): {class_counts[0]}명 ({class_ratio[0]:.1f}%)")
print(f"  이탈 (1): {class_counts[1]}명 ({class_ratio[1]:.1f}%)")
print(f"\n불균형 비율: 1:{imbalance_ratio:.2f}")

In [ ]:
# 테스트
assert class_counts.sum() == len(df), "전체 합이 맞아야 함"
assert abs(class_ratio.sum() - 100) < 0.01, "비율 합이 100%여야 함"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. `value_counts()`로 클래스별 개수 확인
2. `normalize=True`로 비율 계산
3. 다수 클래스 / 소수 클래스로 불균형 비율 계산

**핵심 개념**:
- 클래스 불균형: 한 클래스가 다른 클래스보다 훨씬 많은 상황
- 불균형 시 문제: 모델이 다수 클래스만 예측하는 경향

**실무 팁**:
- 불균형 비율이 1:3 이상이면 처리 고려
- 해결책: stratify, class_weight, SMOTE, 언더샘플링

---

## Q4. 새로운 특성 생성

**문제**: 'monthly_charge'와 'tenure_months'를 사용하여 '예상 총 결제액' 특성을 생성하세요.

In [ ]:
# 정답 코드
df_fe = df.copy()

# 예상 총 결제액 = 월 결제액 * 가입 기간(월)
df_fe['expected_total_charge'] = df_fe['monthly_charge'] * df_fe['tenure_months']

# 실제 총 결제액과 비교
df_fe['charge_diff'] = df_fe['total_charge'] - df_fe['expected_total_charge']

print("새로운 특성 생성 완료!")
print("\n예상 총 결제액 통계:")
print(df_fe['expected_total_charge'].describe())

print("\n실제 vs 예상 결제액 차이 통계:")
print(df_fe['charge_diff'].describe())

In [ ]:
# 테스트
assert 'expected_total_charge' in df_fe.columns, "특성이 생성되어야 함"
assert df_fe['expected_total_charge'].isnull().sum() == 0, "결측치가 없어야 함"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. 도메인 지식 활용: 예상 총 결제액 = 월 결제액 * 가입 기간
2. 추가 특성: 실제 결제액과 예상 결제액의 차이

**핵심 개념**:
- 특성 엔지니어링: 기존 특성을 조합하여 새로운 정보 생성
- 도메인 지식이 중요: 비즈니스 로직 반영

**대안**:
```python
# 월평균 결제액
df_fe['avg_monthly'] = df_fe['total_charge'] / (df_fe['tenure_months'] + 1)
```

---

## Q5. 데이터 분할

**문제**: X, y를 train/test로 분할하세요 (test_size=0.25, stratify 적용, random_state=42).

In [ ]:
# 정답 코드
# 특성 선택 (수치형만)
feature_cols = ['age', 'tenure_months', 'monthly_charge', 'total_charge',
                'num_products', 'complaints', 'support_calls', 'last_purchase_days',
                'avg_order_value', 'order_frequency', 'satisfaction_score']

X = df[feature_cols]
y = df['churned']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

print(f"훈련 데이터: {X_train.shape[0]}개 ({len(X_train)/len(X)*100:.0f}%)")
print(f"테스트 데이터: {X_test.shape[0]}개 ({len(X_test)/len(X)*100:.0f}%)")
print(f"\n훈련 이탈률: {y_train.mean():.1%}")
print(f"테스트 이탈률: {y_test.mean():.1%}")

In [ ]:
# 테스트
assert len(X_train) + len(X_test) == len(X), "분할 후 합이 같아야 함"
assert abs(y_train.mean() - y_test.mean()) < 0.05, "stratify로 비율 유지"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. 특성(X)과 타겟(y) 분리
2. `train_test_split`으로 분할
3. `stratify=y`로 클래스 비율 유지

**핵심 개념**:
- stratify: 분류 문제에서 클래스 비율 유지
- random_state: 재현성 보장

**흔한 실수**:
- stratify 없이 분할 시 테스트셋에 특정 클래스 편중 가능
- random_state 미설정 시 매번 다른 결과

---

## Q6. Logistic Regression 훈련

**문제**: StandardScaler + LogisticRegression 파이프라인을 구성하고 훈련하세요.

In [ ]:
# 정답 코드
# 파이프라인 구성
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000))
])

# 훈련
pipeline.fit(X_train, y_train)

print("파이프라인 훈련 완료!")
print(f"\n파이프라인 구성: {pipeline.named_steps}")

In [ ]:
# 테스트
y_pred = pipeline.predict(X_test)
assert len(y_pred) == len(y_test), "예측 길이가 맞아야 함"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. `Pipeline`으로 전처리와 모델 연결
2. 튜플 리스트로 단계 정의: `(이름, 객체)`
3. `fit`으로 전체 파이프라인 훈련

**핵심 개념**:
- Pipeline: 전처리-모델을 하나의 객체로 관리
- 데이터 누수 방지: fit은 훈련 데이터만

**실무 팁**:
- `max_iter=1000`: 수렴 경고 방지
- 파이프라인 사용 시 코드 간결화 + 오류 감소

---

## Q7. 모델 평가

**문제**: 훈련된 모델의 Accuracy, Precision, Recall, F1, AUC-ROC를 계산하여 출력하세요.

In [ ]:
# 정답 코드
# 예측
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

# 평가 지표 계산
metrics = {
    'Accuracy': accuracy_score(y_test, y_pred),
    'Precision': precision_score(y_test, y_pred),
    'Recall': recall_score(y_test, y_pred),
    'F1 Score': f1_score(y_test, y_pred),
    'AUC-ROC': roc_auc_score(y_test, y_proba)
}

print("모델 평가 결과")
print("=" * 30)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

In [ ]:
# 테스트
assert 0 <= metrics['Accuracy'] <= 1, "Accuracy는 0-1 범위"
assert 0 <= metrics['AUC-ROC'] <= 1, "AUC-ROC는 0-1 범위"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. `predict()`로 클래스 예측
2. `predict_proba()[:, 1]`로 양성 클래스 확률
3. sklearn.metrics의 각 함수로 지표 계산

**핵심 개념**:
- Accuracy: 전체 정확도 (불균형 시 주의)
- Precision: 양성 예측 중 실제 양성 비율
- Recall: 실제 양성 중 탐지 비율
- F1: Precision-Recall 조화 평균
- AUC-ROC: 분류 성능 종합 (확률 기반)

**흔한 실수**:
- `predict_proba`에서 양성 클래스 인덱스 [1] 사용 필수

---

## Q8. 혼동 행렬 시각화

**문제**: 테스트 데이터에 대한 혼동 행렬을 Plotly로 시각화하세요.

In [ ]:
# 정답 코드
# 혼동 행렬 계산
cm = confusion_matrix(y_test, y_pred)

# Plotly로 시각화
fig = px.imshow(
    cm,
    labels=dict(x="예측", y="실제", color="Count"),
    x=['유지 (0)', '이탈 (1)'],
    y=['유지 (0)', '이탈 (1)'],
    text_auto=True,
    title='혼동 행렬 (Confusion Matrix)',
    color_continuous_scale='Blues'
)

fig.update_layout(width=500, height=500)
fig.show()

# 해석
print(f"\n혼동 행렬 해석:")
print(f"  TN (유지 -> 유지): {cm[0][0]}")
print(f"  FP (유지 -> 이탈): {cm[0][1]}")
print(f"  FN (이탈 -> 유지): {cm[1][0]}")
print(f"  TP (이탈 -> 이탈): {cm[1][1]}")

In [ ]:
# 테스트
assert cm.shape == (2, 2), "2x2 행렬이어야 함"
assert cm.sum() == len(y_test), "합이 테스트 데이터 수와 같아야 함"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. `confusion_matrix`로 2x2 행렬 생성
2. `px.imshow`로 히트맵 시각화
3. `text_auto=True`로 값 표시

**핵심 개념**:
- 혼동 행렬 구조: [[TN, FP], [FN, TP]]
- TN/FP/FN/TP 해석 중요

**실무 팁**:
- FP vs FN 비용이 다른 경우 임계값 조정
- 이탈 예측: FN(놓침)이 더 비용이 클 수 있음

---

## Q9. GridSearchCV 튜닝

**문제**: RandomForest에 대해 다음 파라미터로 GridSearchCV를 수행하세요.

In [ ]:
# 정답 코드
# 파라미터 그리드 정의
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10]
}

# GridSearchCV 설정
grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid,
    scoring='roc_auc',
    cv=3,
    n_jobs=-1,
    verbose=1
)

# 스케일링 (RF는 스케일링 필요 없지만 일관성 위해)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 튜닝 실행
grid_search.fit(X_train_scaled, y_train)

print(f"\n최적 파라미터: {grid_search.best_params_}")
print(f"최적 AUC-ROC (CV): {grid_search.best_score_:.4f}")

In [ ]:
# 테스트
assert hasattr(grid_search, 'best_params_'), "튜닝이 완료되어야 함"
assert grid_search.best_score_ > 0.5, "AUC가 0.5보다 커야 함"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. 탐색할 파라미터를 딕셔너리로 정의
2. `GridSearchCV`로 모든 조합 탐색
3. `best_params_`, `best_score_`로 결과 확인

**핵심 개념**:
- GridSearch: 모든 파라미터 조합 시도
- CV (Cross-Validation): 과적합 방지
- scoring: 최적화 기준 지표

**대안**:
- RandomizedSearchCV: 대규모 그리드에 효율적
- Optuna: 베이지안 최적화

---

## Q10. 비즈니스 인사이트 도출

**문제**: 최종 모델을 사용하여 다음을 수행하세요.

In [ ]:
# 정답 코드
# 1. 최적 모델로 전체 고객 예측
best_model = grid_search.best_estimator_
X_all_scaled = scaler.fit_transform(X)
churn_proba = best_model.predict_proba(X_all_scaled)[:, 1]

print("1. 전체 고객 이탈 확률 예측 완료")
print(f"   평균 이탈 확률: {churn_proba.mean():.2%}")
print(f"   최대 이탈 확률: {churn_proba.max():.2%}")

# 2. 이탈 확률 0.5 이상 고객 수
high_risk_count = (churn_proba >= 0.5).sum()
print(f"\n2. 고위험 고객 수 (이탈 확률 >= 50%): {high_risk_count}명")

# 3. 상위 5개 특성 중요도
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': best_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n3. 상위 5개 특성 중요도:")
for _, row in feature_importance.head(5).iterrows():
    print(f"   - {row['feature']}: {row['importance']:.4f}")

# 4. 비즈니스 권장 조치
print("\n4. 비즈니스 권장 조치:")
print("   - 만족도 점수가 낮은 고객 대상 즉시 CS 개선 프로그램 실행")
print("   - 고위험 고객 대상 특별 할인 쿠폰 또는 VIP 혜택 제공")
print("   - 불만 건수가 높은 고객에게 전담 담당자 배정")

In [ ]:
# 테스트
assert len(churn_proba) == len(df), "전체 고객에 대한 예측"
assert high_risk_count >= 0, "고위험 고객 수는 0 이상"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. 전체 데이터에 대해 예측 수행
2. 임계값(0.5) 기준으로 고위험 분류
3. `feature_importances_`로 중요도 분석
4. 분석 결과를 비즈니스 언어로 변환

**핵심 개념**:
- ML 결과를 비즈니스 액션으로 연결
- 특성 중요도로 핵심 이탈 요인 파악
- 임계값 조정으로 리스크 관리

**실무 팁**:
- 경영진 리포트는 기술 용어 최소화
- ROI 관점에서 권장 조치 우선순위화
- 대시보드로 지속적 모니터링

---

## 학습 정리

### 퀴즈 난이도 분포

| 난이도 | 퀴즈 번호 | 주요 내용 |
|--------|----------|----------|
| 기초 | Q1, Q2, Q3 | 프로젝트 정의, 결측치, 클래스 불균형 |
| 응용 | Q4, Q5, Q6 | 특성 생성, 데이터 분할, 파이프라인 |
| 심화 | Q7, Q8 | 모델 평가, 혼동 행렬 시각화 |
| 종합 | Q9, Q10 | GridSearch, 비즈니스 인사이트 |

### 핵심 학습 포인트

1. **End-to-End 파이프라인**: 문제 정의 -> EDA -> 전처리 -> 모델링 -> 평가 -> 배포
2. **특성 엔지니어링**: 도메인 지식 활용이 모델 성능에 큰 영향
3. **모델 비교**: 여러 모델 비교 후 최적 선택
4. **하이퍼파라미터 튜닝**: GridSearchCV로 체계적 최적화
5. **비즈니스 연결**: 모델 결과를 비즈니스 액션으로 변환

### 실무 체크리스트

- [ ] 문제 정의 및 성공 기준 명확화
- [ ] 데이터 품질 확인 (결측치, 이상치, 중복)
- [ ] 클래스 불균형 처리
- [ ] 적절한 평가 지표 선택
- [ ] 여러 모델 비교
- [ ] 하이퍼파라미터 튜닝
- [ ] 특성 중요도 분석
- [ ] 비즈니스 인사이트 도출